# BMTC

> Bengaluru Metropolitan Transport Corporation (BMTC) is a state-owned public road transport corporation in the Indian city of Bengaluru.

In [ ]:
#| default_exp bmtc

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import string
import json
import time
from pathlib import Path

import logging
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s"
)

import requests
import pandas as pd

In [ ]:
data_directory = Path('../data/bmtc/')
data_directory.mkdir(exist_ok=True)

In [ ]:
#| export
def fetch_routes(route_text: str = ""):
    logging.info(f"Fetching routes for route_text = '{route_text}'")
    url = "https://bmtcmobileapi.karnataka.gov.in/WebAPI/SearchRoute_v2"

    headers = {
        "Content-Type": "application/json",
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36",
        "lan": "en"
    }

    if route_text != '':
        payload = {"routetext": route_text}
        try:
            response = requests.post(url, headers=headers, json=payload)
            response.raise_for_status()
            return response.json()['data']
        except requests.exceptions.RequestException as e:
            print(f"Error: {e}")
            return None
    else:
        routes = []
        characters = string.digits + string.ascii_lowercase
        for route_text in characters:
            time.sleep(0.1)
            routes += fetch_routes(route_text)
        return routes

In [ ]:
#| eval: false
#| output: false
routes = fetch_routes()

routes_df = pd.DataFrame(routes)
routes_df = routes_df.drop_duplicates(subset=["routeno"], keep="first")
routes_df.to_csv(data_directory / 'routes.csv')

print(routes_df.shape)
routes_df.head(3)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()